In [7]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd
import numpy as np
import bocd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import yfinance as yf

from machine_learning_finance import (analyze_trades, 
    calc_probabilties_without_lookahead, plot_backtest_analysis, 
    metrics_to_dataframe, create_train_test_windows, make_inverse_env_for,
    calculate_polynomial_regression, TraderEnv, make_price_marker_from_boolean, add_marker, add_line)

import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)
windows = [300, 600, 900, 1500]

symbol = "QQQ"
file = f"../backtests/{symbol}-model-back-test.csv"
period = 365
pd.options.display.max_rows = None
inverse = None # set to anything to graph inverse longs

def plot_win_loss(file, metrics):
    ledger = pd.read_csv(file)
    # Set the style for the plots
    sns.set(style='whitegrid')

    # Create a bar plot for profit and loss stats
    profit_stats, loss_stats = metrics['profit_stats'], metrics['loss_stats']
    profit_loss_df = pd.DataFrame([profit_stats, loss_stats], columns=['min', 'max', 'mean', 'median', 'std'], index=['profit', 'loss'])

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=profit_loss_df.transpose(), palette='muted')
    ax.set_title('Profit and Loss Statistics')
    ax.set_ylabel('Value')

    # Show the plot
    plt.show()


        
def analyze_and_graph_range(file, symbol, start, end, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df, start=start, end=end)    
    test_df = calc_probabilties_without_lookahead(test_df, hist_df, hist_cutoff="2023-05-15")
    plot_backtest_analysis(test_df, ledger, inverse=symbol)
    return ledger, test_df
   
def analyze_and_graph(file, symbol, period, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df, None, 365 * 4, None, 365)
    env = TraderEnv(symbol, test_df, hist_df)
    
    # Todo here: we have all the data we need to render our graphs in the env already
    # We need to modify plot_backtest_analysis to:
    # * render change points
    # * render new moving averages
    # * remove all the probability stuff
    # * Figure out how to supply the polynomials!!!
    
    fig = plot_backtest_analysis(env.orig_timeseries_with_analytics, ledger, inverse=inverse)

    # convert each change point column to price of the average in question
    analytics_df = env.orig_timeseries_with_analytics
    periods=[30, 60, 90]
    colors=["blue", "purple", "green"]
    for period, color in zip(periods, colors):        
        marker_col =  f"cp-loc-{period}"
        trend_col = f"trend-{period}"
        analytics_df = make_price_marker_from_boolean(analytics_df, f"change-point-{period}", f"trend-{period}", marker_col) 
        add_marker(fig, analytics_df, marker_col, f"change-point-{period}", "diamond", 8, color)
        add_line(fig, analytics_df, trend_col, f"moving-avg-{period}", color)
    fig.update_layout(title='Backtest Analysis', xaxis_title='Date', height=800)
    fig.show()

    
    fig = make_subplots(rows=3, cols=1)
    indexes=[1,2,3]
    for period, color, index in zip(periods, colors, indexes):        
        fig.add_trace(go.Bar(x=analytics_df.index, y=analytics_df[f'polynomial_derivative-{period}'], marker=dict(color=f'{color}', colorscale='Viridis'), name=f'Polynomial Derivative {period}'), row=index, col=1)

    fig.update_layout(title='Backtest Analysis', xaxis_title='Date', height=800)
    fig.show()

    return ledger, test_df, env

ticker_obj = yf.download(tickers=symbol)
df = pd.DataFrame(ticker_obj)


ledger, df2, env = analyze_and_graph(file, symbol, 365*4, df)    
#ledger, df2 = analyze_and_graph_range(file, symbol, "03-27-2020", "12-17-2021", df)

# Display metrics as text
metrics = analyze_trades(ledger, symbol, period)
metrics["file"] = file
metrics_df = metrics_to_dataframe(metrics) 
env.orig_timeseries_with_analytics
#df2[["moving_avg", "change_points", "rt", "Close"]]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,trend-30,trend-diff-30,change-point-30,polynomial_derivative-30,...,trend-diff-90,change-point-90,polynomial_derivative-90,long_entry,long_exit,short_entry,short_exit,cp-loc-30,cp-loc-60,cp-loc-90
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-04,354.079987,361.399994,351.970001,358.010010,354.355469,86318300,375.132331,-17.122322,False,-1.205961,...,-22.846657,False,0.087163,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-07,358.619995,361.049988,353.839996,355.130005,351.504883,58198100,373.739331,-18.609326,False,-1.180468,...,-25.680551,False,0.057242,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-08,353.959991,360.239990,352.799988,359.119995,355.454163,64720900,372.260664,-13.140669,False,-1.154975,...,-21.703450,False,0.027322,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-09,363.760010,366.809998,362.119995,366.730011,362.986481,55766400,371.097998,-4.367987,False,-1.129483,...,-14.166212,False,-0.002598,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-10,359.519989,366.489990,356.540009,358.429993,354.771179,112468500,369.660665,-11.230672,False,-1.103990,...,-22.530785,False,-0.032519,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-11,358.429993,360.070007,345.799988,347.059998,343.517212,101569100,367.884331,-20.824334,False,-1.078497,...,-33.786114,False,-0.062439,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-14,346.489990,350.980011,343.890015,347.489990,343.942841,80254400,366.205664,-18.715674,False,-1.053004,...,-33.220788,False,-0.092359,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-15,352.910004,356.369995,351.820007,356.130005,352.494659,55128800,364.687331,-8.557326,False,-1.027512,...,-24.504773,False,-0.122279,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-16,353.820007,357.089996,350.709991,356.040009,352.405609,58947100,363.339665,-7.299656,False,-1.002019,...,-24.537880,False,-0.152200,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ledger